In [51]:
#Importing the needed libraries
import requests
import re
import pandas as pd
import numpy as np


In [3]:
#Testing the API request
headers={'X-Auth-Token':'f886b2fe920f488fba3c87625e94c2ae'}
response=requests.get('https://api.football-data.org/v2/competitions/CL/matches?season=2018',headers=headers)
championsleague=response.json()

In [5]:
#Function to extract matches data from API
def get_matches(year):
    response=requests.get('https://api.football-data.org/v2/competitions/CL/matches?season={}'.format(year),headers=headers)
    CL=response.json()
    awayt=[]
    homet=[]
    homeg=[]
    awayg=[]
    stage=[]
    Date=[]
    ref=[]
    for match in CL['matches']:
      awayt.append(match['awayTeam']['name'])
      homet.append(match['homeTeam']['name'])
      homeg.append(match['score']['fullTime']['homeTeam'])
      awayg.append(match['score']['fullTime']['awayTeam'])
      stage.append(match['stage'])
      Date.append(match['utcDate'])
      

    #creating the dataframe
    data=pd.DataFrame({'Home team':homet,'Away team':awayt,
                         'Home goals':homeg,'Away goals':awayg,'Stage':stage,
                         'date':Date})
    
    return data


  


In [6]:
#Building a dataframe of the last 3 seasons
CL_matches=pd.DataFrame(columns=['Home team','Away team','Home goals','Away goals','Stage','date'])
for year in [2018,2019,2020]:
  CL_matches=CL_matches.append(get_matches(year))
  
  CL_matches

In [7]:
#Cleaning the matches dataframe
def CL_clean(data):
  
  data.fillna(0,inplace=True)
  data.replace('TBP',0,inplace=True)
  data['Away goals']=data['Away goals'].astype('int64')
  data['Home goals']=data['Home goals'].astype('int64')
  data['Stage']=data['Stage'].str.replace('_',' ').str.title()
  data['date']=pd.to_datetime(data['date'],dayfirst=True).dt.date
  
  return data

In [8]:
CL_matches=CL_clean(CL_matches)
CL_matches

,Home team,Away team,Home goals,Away goals,Stage,date
0,FC Santa Coloma,Drita KF Gjilan,0,2,Preliminary Semi Finals,2018-06-26
1,SP La Fiorita,Lincoln Red Imps FC,0,2,Preliminary Semi Finals,2018-06-26
2,Lincoln Red Imps FC,Drita KF Gjilan,1,4,Preliminary Final,2018-06-29
3,FC Torpedo Kutaisi,FC Sheriff Tiraspol,2,1,1St Qualifying Round,2018-07-10
4,FC Flora,Hapoel Be'er Sheva FC,1,4,1St Qualifying Round,2018-07-10
...,...,...,...,...,...,...
173,Real Madrid CF,Chelsea FC,1,1,Semi Finals,2021-04-27
174,Paris Saint-Germain FC,Manchester City FC,1,2,Semi Finals,2021-04-28
175,Manchester City FC,Paris Saint-Germain FC,2,0,Semi Finals,2021-05-04
176,Chelsea FC,Real Madrid CF,2,0,Semi Finals,2021-05-05


In [9]:
#Function which returns the winner of every match
def winner(CL_matches):
  if CL_matches['Home goals']>CL_matches['Away goals']:
    return CL_matches['Home team']
  elif CL_matches['Home goals']<CL_matches['Away goals']:
    return CL_matches['Away team']
  else:
    return 'Draw'

   

In [10]:
CL_matches['Match winner']=CL_matches.apply(winner,axis=1)
CL_matches

,Home team,Away team,Home goals,Away goals,Stage,date,Match winner
0,FC Santa Coloma,Drita KF Gjilan,0,2,Preliminary Semi Finals,2018-06-26,Drita KF Gjilan
1,SP La Fiorita,Lincoln Red Imps FC,0,2,Preliminary Semi Finals,2018-06-26,Lincoln Red Imps FC
2,Lincoln Red Imps FC,Drita KF Gjilan,1,4,Preliminary Final,2018-06-29,Drita KF Gjilan
3,FC Torpedo Kutaisi,FC Sheriff Tiraspol,2,1,1St Qualifying Round,2018-07-10,FC Torpedo Kutaisi
4,FC Flora,Hapoel Be'er Sheva FC,1,4,1St Qualifying Round,2018-07-10,Hapoel Be'er Sheva FC
...,...,...,...,...,...,...,...
173,Real Madrid CF,Chelsea FC,1,1,Semi Finals,2021-04-27,Draw
174,Paris Saint-Germain FC,Manchester City FC,1,2,Semi Finals,2021-04-28,Manchester City FC
175,Manchester City FC,Paris Saint-Germain FC,2,0,Semi Finals,2021-05-04,Manchester City FC
176,Chelsea FC,Real Madrid CF,2,0,Semi Finals,2021-05-05,Chelsea FC


In [59]:
#Provides each team's statistiques
def team_stat(team):
   
   data=CL_matches[(CL_matches['Home team']==team) | (CL_matches['Away team']==team)]
   winning_perct=100*(len(data[data['Match winner']==team])/len(data))
   matches_played=len(data)
   goals_for_home=sum(data[data['Home team']==team]['Home goals'])
   goals_for_away=sum(data[data['Away team']==team]['Away goals'])
   goals_against_home=sum(data[data['Home team']!=team]['Home goals'])
   goals_against_away=sum(data[data['Away team']!=team]['Away goals'])
   goals_for=goals_for_home+goals_for_away
   goals_against=goals_against_home+goals_against_away
   
   return winning_perct,goals_for,goals_against,matches_played



In [63]:
stats=[]
for team in CL_matches['Home team'].unique():
  stats.append(team_stat(team))
stats=np.array(stats) 


In [73]:
team_stats=pd.DataFrame({'Team':CL_matches['Home team'].unique(),
                         'Winning Percentage':stats[:,0].round(2),
                         'Goals for':stats[:,1],
                         'Goals against':stats[:,2],
                         'Matches Played':stats[:,3]} )
team_stats

,Team,Winning Percentage,Goals for,Goals against,Matches Played
0,FC Santa Coloma,33.33,2.0,4.0,3.0
1,SP La Fiorita,0.00,0.0,2.0,1.0
2,Lincoln Red Imps FC,33.33,3.0,5.0,3.0
3,FC Torpedo Kutaisi,50.00,2.0,4.0,2.0
4,FC Flora,0.00,3.0,8.0,3.0
...,...,...,...,...,...
122,Stade Rennais FC 1901,0.00,3.0,11.0,6.0
123,SS Lazio,25.00,13.0,13.0,8.0
124,Borussia Mönchengladbach,25.00,16.0,13.0,8.0
125,Olympique de Marseille,16.67,2.0,13.0,6.0


In [14]:
#testting the api by requesting top scorere's data of 2018
response=requests.get('https://api.football-data.org/v2/competitions/CL/scorers?limit=20&season=2018',headers=headers)
comp=response.json()

In [15]:
def get_scorers(year):
  response=requests.get('https://api.football-data.org/v2/competitions/CL/scorers?limit=30&season={}'.format(year),headers=headers)
  scorer=response.json()
  name=[]
  nationality=[]
  position=[]
  goals=[]
  birthdate=[]
  team=[]
  for player in scorer['scorers']:
    name.append(player['player']['name'])
    nationality.append(player['player']['nationality'])
    position.append(player['player']['position'])
    goals.append(player['numberOfGoals'])
    birthdate.append(player['player']['dateOfBirth'])
    team.append(player['team']['name'])
  

  
  data=pd.DataFrame({'Player Name':name,'Nationality':nationality,
                         'Position':position,'Number of goals':goals,'Team':team,'birth date':birthdate})
  data['age']=pd.to_datetime(data['birth date'])
  data['age']=pd.to_datetime('today').year-pd.DatetimeIndex(data['age']).year                    
    
  return data




  




In [17]:
#Building dataframe of top scorers for last 3 seasons

Cl_scorers=pd.DataFrame(columns=['Player Name','Nationality','Position','Number of goals','Team','birth date','age'])
for year in [2018,2019,2020]:
   Cl_scorers=Cl_scorers.append(get_scorers(year))
Cl_scorers1=Cl_scorers.groupby(['Player Name'],as_index=False)[['Number of goals']].sum()
Cl_scorers1=Cl_scorers1.merge(Cl_scorers,how='left' ,on='Player Name',copy=False)
Cl_scorers1=Cl_scorers1.sort_values(by=['Number of goals_x'],ascending=False)
Cl_scorers1.drop('Number of goals_y',axis=1,inplace=True)
Cl_scorers1.drop_duplicates(inplace=True)
Cl_scorers1.rename(columns={'Number of goals_x':'Number of goals'},inplace=True)
Cl_scorers1.reset_index(drop=True,inplace=True)

In [18]:
Cl_scorers1

,Player Name,Number of goals,Nationality,Position,Team,birth date,age
0,Robert Lewandowski,28,Poland,Attacker,FC Bayern München,1988-08-21,33
1,Erling Haaland,20,Norway,Attacker,Borussia Dortmund,2000-07-21,21
2,Lionel Messi,17,Argentina,Attacker,FC Barcelona,1987-06-24,34
3,Kylian Mbappé,17,France,Attacker,Paris Saint-Germain FC,1998-12-20,23
4,Cristiano Ronaldo,14,Portugal,Attacker,Juventus FC,1985-02-05,36
...,...,...,...,...,...,...,...
72,Franck Boli,3,Côte d’Ivoire,Attacker,Ferencvárosi TC,1993-12-07,28
73,Myrto Uzuni,3,Albania,Attacker,Ferencvárosi TC,1995-05-31,26
74,Anders Dreyer,3,Denmark,Attacker,FC Midtjylland,1998-05-02,23
75,Rémy Cabella,3,France,Midfielder,FK Krasnodar,1990-03-08,31


In [77]:
Cl_scorers1.to_csv('top_scorer.csv')

In [78]:
team_stats.to_csv('stats.csv')
CL_matches.to_csv('CL matches.csv')